In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
# import sys
# sys.path.append("/root/sae_auto_interp")
from sae_auto_interp.explainers import SimpleExplainer
from sae_auto_interp.features import FeatureDataset, FeatureLoader
from sae_auto_interp.config import FeatureConfig
from sae_auto_interp.clients import Local

import os
from nnsight import LanguageModel
from simple_parsing import ArgumentParser
from load_sae_from_hf import load_sae_from_hf
from sae_auto_interp.autoencoders import load_saelens_autoencoders
from sae_auto_interp.config import CacheConfig
from sae_auto_interp.features import FeatureCache
from sae_auto_interp.utils import load_filter, load_tokenized_data
from nnsight import LanguageModel

ImportError: cannot import name 'FeatureLoader' from 'sae_auto_interp.features' (/root/sae-auto-interp/sae_auto_interp/features/__init__.py)

In [5]:
# model = LanguageModel("google/gemma-2b-it", device_map="cpu", dispatch=True)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# get tokenizer from HF
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

In [5]:
cfg = FeatureConfig()
dataset = FeatureDataset(
    raw_dir="/root/sae-auto-interp/splits/.model.layers.12",
    cfg=cfg,
)

In [ ]:
loader = FeatureLoader(
    tokens=tokens,
    dataset=dataset,
    constructor = # constructor,
)

In [6]:
client = Local("casperhansen/llama-3-70b-instruct-awq", base_url=f"http://localhost:8000/v1")

In [9]:
import orjson
from sae_auto_interp.pipeline import process_wrapper, Pipeline
explanation_dir = "/root/sae-auto-interp/explanations"
if not os.path.isdir(explanation_dir):
    os.makedirs(explanation_dir)

def explainer_postprocess(result):
    with open(f"{explanation_dir}/{result.record.feature}.txt", "wb") as f:
        f.write(orjson.dumps(result.explanation))

    return result

explainer_pipe = process_wrapper(
    SimpleExplainer(
        client,
        tokenizer=tokenizer,
    ),
    postprocess=explainer_postprocess,
)

In [ ]:
import asyncio
pipeline = Pipeline(
    loader.load,
    explainer_pipe,
)

asyncio.run(pipeline.run())